In [41]:
import numpy as np
from helpers import *

from implementations import *
from preprocess import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
variables = {
    "categorical_vars": [
    "SEX",
    "_RFHLTH",
    "_HCVU651",
    "_RFHYPE5",
    "_CHOLCHK",
    "_RFCHOL",
    "_MICHD",
    "_LTASTH1",
    "_CASTHM1",
    "_ASTHMS1",
    "_DRDXAR1",
    "_PRACE1",
    "_MRACE1",
    "_M_RACE",
    "_HISPANC",
    "_RACE",
    "_RACEG21",
    "_RACEGR3",
    "_RACE_G1",
    "_AGEG5YR",
    "_AGE_G",
    "_BMI5CAT",
    "_RFBMI5",
    "_CHLDCNT",
    "_INCOMG",
    "_SMOKER3",
    "_RFSMOK3",
    "DRNKANY5",
    "_RFDRHV5",
    "_FRTLT1",
    "_VEGLT1",
    "_TOTINDA",
    "ACTIN11_",
    "ACTIN21_",
    "_PACAT1",
"_PAINDX1",
"_PA150R2",
"_PA300R2",
"_PA30021",
"_PASTRNG",
"_PAREC1",
"_LMTACT1",
"_LMTWRK1",
"_AIDTST3"
],
    "numerical_vars":["AGE", "HTIN4","HTM4","WTKG3","_BMI5","_DRNKWEK", "FTJUDA1_", "GRENDAY_","BEANDAY_","FRUTDA1_","ORNGDAY_","VEGEDA1_","PADUR1_","PADUR2_","_MINAC11"]
}
    
data_path="./data/dataset_to_release"
x_train, x_test, y_train, train_ids, test_ids, selected_indices_categorical_train = load_csv_data(data_path, selected_cols=variables)

In [45]:
x_train.shape[1]

56

In [46]:
x_undersampled_train, y_undersampled_train = undersample(x_train, y_train)

In [114]:
all_categories = [np.unique(x_undersampled_train[:, idx][~np.isnan(x_undersampled_train[:, idx])]) for idx in selected_indices_categorical_train]

full_data_preprocessed, _ = apply_preprocessing(x_undersampled_train, x_test, selected_indices_categorical_train, all_categories)
if np.isnan(full_data_preprocessed).any():
    print("NaNs introduced during preprocessing on the full dataset!")


Detected NaNs in column 5. Imputing with value 2.0.
Detected NaNs in column 9. Imputing with value 2.0.
Detected NaNs in column 16. Imputing with value 1.0.
Detected NaNs in column 19. Imputing with value 3.0.
Detected NaNs in column 37. Imputing with value 3.0.
Detected NaNs in column 38. Imputing with value 3.0.
Detected NaNs in column 39. Imputing with value 2.0.
Detected NaNs in column 40. Imputing with value 66.0.
Detected NaNs in column 41. Imputing with value 1.68.
Detected NaNs in column 42. Imputing with value 90.72.
Detected NaNs in column 43. Imputing with value 26.63.
Detected NaNs in column 45. Imputing with value 0.0.
Detected NaNs in column 46. Imputing with value 1.0.
Detected NaNs in column 47. Imputing with value 0.0.
Detected NaNs in column 48. Imputing with value 1.0.
Detected NaNs in column 49. Imputing with value 0.0.
Detected NaNs in column 50. Imputing with value 1.0.
Detected NaNs in column 5. Imputing with value 1.0.
Detected NaNs in column 9. Imputing with va

In [47]:
x_undersampled_train.shape[1]

56

In [48]:
# Check the distribution of the _MICHD labels
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

unique, counts = np.unique(y_undersampled_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{-1: 299160, 1: 28975}
{-1: 28975, 1: 28975}


In [49]:
x_undersampled_train, dropped_train_cols = drop_high_nan_columns(x_undersampled_train)
x_test = np.delete(x_test, dropped_train_cols, axis=1)

In [50]:
x_undersampled_train.shape[1]

51

In [51]:
def compute_metrics(y_true, y_pred):
    """
    Compute precision, recall, accuracy, and F1 score.
    
    Parameters:
    - y_true: List of true labels
    - y_pred: List of predicted labels
    
    Returns:
    - Dictionary with precision, recall, accuracy, and F1 score
    """
    TP = sum([1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 1])
    FP = sum([1 for yt, yp in zip(y_true, y_pred) if yt == -1 and yp == 1])
    FN = sum([1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == -1])
    TN = sum([1 for yt, yp in zip(y_true, y_pred) if yt == -1 and yp == -1])
    
    # Calculate metrics
    precision_val = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall_val = TP / (TP + FN) if (TP + FN) != 0 else 0
    accuracy_val = (TP + TN) / len(y_true)
    f1_val = 2 * (precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) != 0 else 0
    
    return {
        "Precision": precision_val,
        "Recall": recall_val,
        "Accuracy": accuracy_val,
        "F1 Score": f1_val
    }


In [122]:
def build_stratified_k_indices_with_seed(y, k_fold, seed=42):
    """
    Randomly partitions the indices of the dataset into k groups in a stratified manner using a fixed seed.
    Args:
        y: labels, used for indexing
        k_fold: number of groups after the partitioning
        seed: the random seed value, default is 42
    Returns:
        k_indices: an array of k sub-indices that are stratified partitioned
    """
    np.random.seed(seed)
    
    # Get unique classes and their counts
    classes, class_counts = np.unique(y, return_counts=True)
    
    # Calculate how many samples of each class should be in each fold
    n_samples_per_class_per_fold = (class_counts / k_fold).astype(int)
    n_samples_remaining = class_counts % k_fold

    # Distribute samples for each class to the folds
    k_indices = [[] for _ in range(k_fold)]
    
    for c in classes:
        class_indices = np.where(y == c)[0]
        np.random.shuffle(class_indices)
        
        fold_counts = n_samples_per_class_per_fold[c] * np.ones(k_fold, dtype=int)
        fold_counts[:n_samples_remaining[c]] += 1  # Distribute the remaining samples
        
        current = 0
        for k in range(k_fold):
            start, stop = current, current + fold_counts[k]
            k_indices[k].extend(class_indices[start:stop])
            current = stop

    # Convert each fold's indices to a numpy array
    k_indices = [np.array(indices) for indices in k_indices]

    return np.array(k_indices)


In [140]:
def prepare_data_for_fold(y, x, k_indices, k, selected_indices_categorical_train, all_categories):
    """
    Prepare the data for a given fold index.

    Args:
        y: Vector of labels
        x: Feature matrix
        k_indices: 2D array returned by build_k_indices()
        k: scalar, the k-th fold
        degree: Degree for polynomial expansion
        high_skewness_cols: Columns that need log transform
        high_residual_indices: Columns that need polynomial expansion

    Returns:
        train_x, train_y, test_x, test_y: Prepared data for the fold
    """
    
    test_x, test_y = x[k_indices[k]], y[k_indices[k]]
    train_x, train_y = (
        x[k_indices[(np.arange(len(k_indices)) != k)].reshape(-1)],
        y[k_indices[(np.arange(len(k_indices)) != k)].reshape(-1)],
    )
    
    #print(f"Shape of train_x before preprocessing: {train_x.shape}")
    #print(f"Shape of test_x before preprocessing: {test_x.shape}")
    
    train_x, test_x = apply_preprocessing(train_x, test_x, selected_indices_categorical_train, all_categories)
    #print(f"Shape of train_x after preprocessing: {train_x.shape}")
    #print(f"Shape of test_x after preprocessing: {test_x.shape}")
    if np.isnan(train_x).any():
        print(f"NaNs detected in train_x after initial k-fold splitting for fold {k}!")
    if np.isnan(test_x).any():
        print(f"NaNs detected in test_x after initial k-fold splitting for fold {k}!")

    
    return train_x, train_y, test_x, test_y

In [141]:
def predict(data, weights, is_logistic):
        preds = data.dot(weights)
        if is_logistic:
            preds = np.where(sigmoid(preds) > 0.5, 1, -1)
        else:
            preds = np.where(preds >= 0, 1, -1)
        return preds

In [146]:
def train_and_evaluate(train_x, train_y, test_x, test_y, model_func, 
                       lambda_=0.1, max_iters=1000, gamma=0.01):
    """
    Train a model and evaluate it. Now adapted for weighted training and flexible arguments.

    Args:
        train_x, train_y: Training data
        test_x, test_y: Testing data
        model_func: The regression function to use
        weights: Sample weights
        lambda_: Regularization parameter (if applicable)
        max_iters: Maximum number of iterations for iterative optimization methods, default 1000.
        gamma: Learning rate for optimization methods, default 0.01.

    Returns:
        Training and testing losses, accuracy, precision, recall, and F1 score
    """
    
    initial_w = np.zeros(train_x.shape[1])
    is_logistic = model_func in [logistic_regression, reg_logistic_regression]

    unique, counts = np.unique(train_y, return_counts=True)
    value_counts = dict(zip(unique, counts))
    #print(value_counts)
    print("***********************")
    train_y_adjusted = (train_y + 1) / 2 if is_logistic else train_y
    
    # --- DIAGNOSTIC CODE INSERTED --- #
    """
    # 1. Check for NaNs in the train data
    if np.any(np.isnan(train_x)):
        print("Warning: NaN values detected in train_x!")
    
    # 2. Check if features are scaled
    feature_means = np.mean(train_x, axis=0)
    feature_stds = np.std(train_x, axis=0)
    print(f"Feature Means: {feature_means}")
    print(f"Feature Stds: {feature_stds}")
    
    # 3. Check for constant features
    constant_features = np.all(train_x == train_x[0, :], axis=0)
    if np.any(constant_features):
        print(f"Warning: Constant features detected at indices {np.where(constant_features)}!")
    
    # --- END OF DIAGNOSTIC CODE --- #
    """
    # Train model based on selected function
    if model_func in [mean_squared_error_gd, mean_squared_error_sgd, logistic_regression]:
        w, loss_history = model_func(train_y_adjusted, train_x, initial_w, max_iters, gamma)
    elif model_func in [reg_logistic_regression]:
        w, loss_history = model_func(train_y_adjusted, train_x, lambda_, initial_w, max_iters, gamma)
    elif model_func in [least_squares, ridge_regression]:
        w, _ = model_func(train_y_adjusted, train_x, lambda_)
    else:
        raise ValueError("Invalid model selection")
    
    # If you have a loss history, print its last few values
    if 'loss_history' in locals() :
        print(f"loss value: {loss_history}")
        print("***********************")

    # Predict and compute metrics for training data
    preds_train = predict(train_x, w, is_logistic)
    loss_train = np.sqrt(np.mean((train_y_adjusted - preds_train)**2))
    metrics_train = compute_metrics(train_y_adjusted, preds_train)
    if np.unique(preds_train).size == 1 and (preds_train[0] == 1 or preds_train[0] == -1):
        print("Warning: Model only assigns", preds_train[0], "to all training samples!")

    # Predict and compute metrics for testing data
    preds_test = predict(test_x, w, is_logistic)
    loss_test = np.sqrt(np.mean((test_y - preds_test)**2))
    metrics_test = compute_metrics(test_y, preds_test)

    if np.unique(preds_test).size == 1 and (preds_test[0] == 1 or preds_test[0] == -1):
        print("Warning: Model only assigns", preds_test[0], "to all test samples!")
        
    return {
        "loss_train": loss_train,
        "loss_test": loss_test,
        "metrics_train": metrics_train,
        "metrics_test": metrics_test
    }


In [147]:
def grid_search_hyperparameters_weighted(y, x, k_indices, model_funcs, categorical_columns_indices):
    """Perform grid search with progress tracking."""
    gammas = [0.001, 0.01, 0.1]
    lambdas = [0.001, 0.01, 0.1]
    max_iters_values = [100, 500]
    
    best_accuracy = 0
    best_f1 = 0
    best_hyperparameters = None
    best_model = None
    all_categories = [np.unique(x[:, idx][~np.isnan(x[:, idx])]) for idx in categorical_columns_indices]

    total_models = len(model_funcs)
    for idx, model_func in enumerate(model_funcs):
        print(f"Evaluating Model {idx+1}/{total_models}: {model_func.__name__}")
        for gamma in gammas:
            for lambda_ in lambdas:
                for max_iters in max_iters_values:
                    fold_accuracies = []
                    fold_f1_scores = []
                    fold_precision = []
                    fold_recall = []
                    for k in range(len(k_indices)):
                        train_x, train_y, test_x, test_y = prepare_data_for_fold(y, x, k_indices, k, selected_indices_categorical_train, all_categories)
                        
                        if np.any(np.isnan(train_x)):
                            print("Warning: NaN values detected in train_x! - althogh imputed pred")
                        results = train_and_evaluate(train_x, train_y, test_x, test_y, model_func, lambda_=lambda_, max_iters=max_iters, gamma=gamma)
                        
                        fold_accuracies.append(results["metrics_test"]["Accuracy"])
                        fold_f1_scores.append(results["metrics_test"]["F1 Score"])
                        fold_precision.append(results["metrics_test"]["Precision"])
                        fold_recall.append(results["metrics_test"]["Recall"])
                    
                    average_accuracy = np.mean(fold_accuracies)
                    average_f1 = np.mean(fold_f1_scores)
                    average_precision = np.mean(fold_precision)
                    average_recall = np.mean(fold_recall)
                    
                    print(f"Gamma: {gamma}, Lambda: {lambda_}, Max Iters: {max_iters}")
                    print(f"Average Accuracy: {average_accuracy:.4f}, Average F1: {average_f1:.4f}")
                    print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}")
                    print("-------------------------------------------------------------")

                    # Update best hyperparameters if this combination gives better F1 score
                    if average_f1 > best_f1:
                        best_f1 = average_f1
                        best_accuracy = average_accuracy
                        best_hyperparameters = {
                            "gamma": gamma,
                            "lambda_": lambda_,
                            "max_iters": max_iters
                        }
                        best_model = model_func.__name__

    return best_model, best_hyperparameters, best_accuracy, best_f1


In [148]:
def apply_best_model(x_train, y_train, x_test, best_model_func, best_hyperparameters):
    """
    Train the best model using the entire training set and get predictions for the test set.
    
    Args:
    - x_train: Training data features
    - y_train: Training data labels
    - x_test: Test data features
    - best_model_func: The best model function obtained from grid search
    - best_hyperparameters: Dictionary containing the best hyperparameters
    
    Returns:
    - preds_test: Predictions for the test data
    """
    
    is_logistic = best_model_func in [logistic_regression, reg_logistic_regression]
    y_train_adjusted = (y_train + 1) / 2 if is_logistic else y_train
    
    initial_w = np.zeros(x_train.shape[1])
    gamma = best_hyperparameters["gamma"]
    lambda_ = best_hyperparameters["lambda_"]
    max_iters = best_hyperparameters["max_iters"]
    
    # Train model using the entire training data
    if best_model_func in [mean_squared_error_gd, mean_squared_error_sgd, logistic_regression]:
        w, _ = best_model_func(y_train_adjusted, x_train, initial_w, max_iters, gamma)
    elif best_model_func in [reg_logistic_regression]:
        w, _ = best_model_func(y_train_adjusted, x_train, lambda_, initial_w, max_iters, gamma)
    elif best_model_func in [least_squares, ridge_regression]:
        w, _ = best_model_func(y_train_adjusted, x_train, lambda_)
    else:
        raise ValueError("Invalid model selection")
    
    # Predict for test data
    preds_test = predict(x_test, w, is_logistic)
    
    return preds_test

In [149]:
k_fold = 5  # for 5-fold cross-validation
seed = 42  
k_indices = build_stratified_k_indices_with_seed(y_undersampled_train, k_fold, seed)
model_funcs = [logistic_regression, reg_logistic_regression]
best_model, best_hyperparameters, best_accuracy, best_f1 = grid_search_hyperparameters_weighted(y_undersampled_train, x_undersampled_train,  k_indices, model_funcs, selected_indices_categorical_train)
print("Best Hyperparameters:", best_hyperparameters)
print("Best Accuracy:", best_accuracy)
print("Best F1 Score:", best_f1)

Evaluating Model 1/2: logistic_regression
{-1: 23180, 1: 23180}
***********************
loss value: 37.112750549587986
{-1: 23180, 1: 23180}
***********************
loss value: 37.600442409823486


KeyboardInterrupt: 